# 1 课堂示例代码

## 1.1 微博数据准备及库引入

In [1]:
# 库引入
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [2]:
# 读取excel文件
df = pd.read_excel('text_analysis_weibo_sample.xlsx', index_col = 0)
df.head()

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东


## 1.2 语料预处理

### 1.2.1 剔除符号与数字

In [3]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['标题/微博内容'][0]
remove_nums(test)[:100]

'国债地产行业重磅利好提振风险偏好期债低开低走国债期货全线收跌年期主力合约跌年期主力合约跌年期主力合约跌三大主力合约均创逾一年收盘新低行情解读公开市场方面央行开展了亿元天期逆回购操作中标利率亿元逆回购到'

### 1.2.2 分词

In [4]:
# 加载中文停用词词典，可个性化设置
# stopwords = open('stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    # words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['标题/微博内容'][0]
clean_text(test)[:100]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\14368\AppData\Local\Temp\jieba.cache
Loading model cost 0.627 seconds.
Prefix dict has been built successfully.


'    国债 ： 地产 行业 重磅 利好 提振 风险 偏好 ， 期债 低开 低 走           国债 期货 全线 收跌 ， 10 年期 主力 合约 跌 0.37% ， 5 年期 主力 合约 跌'

In [5]:
# 数据类型转换和处理
df['标题/微博内容'] = df['标题/微博内容'].astype(str)
df['微博内容分词'] = df['标题/微博内容'].apply(remove_nums)
df['微博内容分词'] = df['微博内容分词'].apply(clean_text)
df['微博内容分词'] = df['微博内容分词'].apply(lambda x: x.split())
df

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他,"[国债, 地产, 行业, 重磅, 利好, 提振, 风险, 偏好, 期债, 低开, 低, 走,..."
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西,"[喜迎, 二十大, 忠诚, 保平安]"
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建,"[注意, 事关, 明日, 教资, 考试, 福建省, 教育, 考试院, 发布, 补充, 公告,..."
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京,"[近日, 千年, 大计, 雄安, 新区, 迎来, 五周岁, 生日, 从, 一张白纸, 到, ..."
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东,"[樊振东, 牛, 逼]"
...,...,...,...,...,...,...,...,...,...,...
95,14034,老公好漂亮[舔屏],0,0,0,aa7af97ae98cbe8983e559b948ecfabf,825,191,北京,"[老公, 好, 漂亮, 舔, 屏]"
96,35249,恭喜@张小娜呀娜 1名用户获得【小飞T】。C官方唯一抽奖工具@C抽奖平台 对本次抽奖进行监督...,0,0,2,f538513e5801c275cbcf285517a8ee62,51709,20,北京,"[恭喜, 张小娜, 呀, 娜, 名, 用户, 获得, 小飞, 官方, 唯一, 抽奖, 工具,..."
97,39598,接不动了，居民部门负债率提升空间太有限了。现在动辄几万一平米，总价高，月供高，预期转差，还怎...,0,0,0,2e76c39b6665a0b6f5bdc4c35f36f573,0,59,四川,"[接不动, 了, 居民, 部门, 负债率, 提升, 空间, 太, 有限, 了, 现在, 动辄..."
98,48977,目前电子烟政策已从制定阶段逐渐走向实施阶段，而主要的电子烟公司股价下跌幅度高达70%-...,0,0,0,NaN,0,0,北京,"[目前, 电子, 烟, 政策, 已, 从, 制定, 阶段, 逐渐, 走向, 实施, 阶段, ..."


## 1.3 LDA模型训练

In [6]:
# 创建词典和语料库
dictionary = corpora.Dictionary(df['微博内容分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['微博内容分词']]     # 根据分词结果创建语料库

In [7]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [8]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.039*"的" + 0.020*"电子" + 0.020*"烟" + 0.008*"在" + 0.007*"年"')
(1, '0.020*"的" + 0.015*"中国" + 0.010*"经济" + 0.005*"世界" + 0.004*"你"')
(2, '0.040*"的" + 0.011*"在" + 0.009*"月" + 0.007*"郑州" + 0.007*"了"')
(3, '0.026*"的" + 0.008*"等" + 0.008*"数字" + 0.007*"了" + 0.007*"年"')
(4, '0.016*"的" + 0.014*"月" + 0.013*"万吨" + 0.008*"市场" + 0.008*"日"')


In [9]:
# 打印第一条主题和得分
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9994240999221802	 
Topic: 0.040*"的" + 0.011*"在" + 0.009*"月" + 0.007*"郑州" + 0.007*"了" + 0.006*"也" + 0.006*"和" + 0.005*"是" + 0.005*"加息" + 0.005*"市场"


In [10]:
# 从DataFrame中提取“标题分词”列的值
documents = df['微博内容分词'].values

In [11]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(2, 0.99942416)]

Document 2:
[(0, 0.83987623), (1, 0.04003867), (2, 0.040028937), (3, 0.04002975), (4, 0.04002641)]

Document 3:
[(4, 0.9973734)]

Document 4:
[(3, 0.99933434)]

Document 5:
[(0, 0.05005098), (1, 0.050728776), (2, 0.79913044), (3, 0.050047643), (4, 0.050042145)]

Document 6:
[(0, 0.9081428), (1, 0.022295766), (2, 0.023064565), (3, 0.022486083), (4, 0.024010792)]

Document 7:
[(0, 0.9495511), (1, 0.012666483), (2, 0.012613777), (3, 0.012573647), (4, 0.012594982)]

Document 8:
[(0, 0.10025845), (1, 0.100386955), (2, 0.10086787), (3, 0.100148015), (4, 0.5983387)]

Document 9:
[(1, 0.9721353)]

Document 10:
[(2, 0.9993365)]



## 1.4 LDA可视化

In [12]:
# 可视化及界面显示
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) # 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
pyLDAvis.display(lda_vis)

# 2 代码练习-智慧养老政策

## 2.1 智慧养老政策数据准备及库引入

In [13]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [14]:
# 读取CSV文件
df = pd.read_csv('智慧养老政策.csv', encoding='gbk')
df

,序号,标题,时效性,效力位阶,制定机关,发文字号,公布日期,施行日期,批准部门,批准日期,原文链接
0,地方规范性文件_672,黑龙江省人民政府办公厅关于推进养老服务发展的实施意见,现行有效,地方规范性文件,黑龙江省民政厅,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/b18e6be8f51692a1cbd...
1,地方规范性文件_673,广西壮族自治区大数据发展局关于全面推进数字化转型发展的意见,现行有效,地方规范性文件,广西壮族自治区大数据发展局,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/22f0c8beef6908e6866...
2,地方规范性文件_674,中共无锡市委、无锡市人民政府印发《＜关于加快推进数字经济高质量发展的实施意见＞＜关于加快推进...,现行有效,地方规范性文件,无锡市人民政府,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/42b6a5f466eaa99be8e...
3,地方规范性文件_675,贵阳市民政局关于推进贵阳贵安养老服务高质量发展的实施意见,现行有效,地方规范性文件,贵阳市民政局,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/e67d28d08fea8084a66...
4,地方规范性文件_676,廊坊市人民政府办公室关于加快推进养老服务体系建设的实施意见,现行有效,地方规范性文件,廊坊市人民政府,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/909deb5c2273a94ef8b...
...,...,...,...,...,...,...,...,...,...,...,...
2777,地方工作文件_15,郑州市人民政府关于印发郑州市数字政府建设行动方案的通知,现行有效,地方工作文件,郑州市人民政府,郑政〔2023〕25号,2023.12.25,2023.12.25,NaN,NaN,https://www.pkulaw.com/lar/b7908e42b1682d8bafd...
2778,部门规章_1,产业结构调整指导目录(2024年本),尚未施行,部门规章,国家发展和改革委员会(含原国家发展计划委员会、原国家计划委员会),中华人民共和国国家发展和改革委员会令第7号,2023.12.27,2024.02.01,NaN,NaN,https://www.pkulaw.com/chl/d8513a2dd8e4e297bdf...
2779,地方工作文件_14,山西省人民政府办公厅关于印发山西省数字政府建设规划(2023―2025年)的通知,现行有效,地方工作文件,山西省人民政府,晋政办发〔2023〕82号,2023.12.28,2023.12.28,NaN,NaN,https://www.pkulaw.com/lar/cf041968e90d5c7d666...
2780,地方工作文件_13,山东省人民政府办公厅关于印发《山东省养老服务高质量发展三年行动计划(2024—2026年)》的通知,现行有效,地方工作文件,山东省人民政府,鲁政办字〔2023〕207号,2023.12.30,2023.12.30,NaN,NaN,https://www.pkulaw.com/lar/d0f5a014d1a25e2fc83...


## 2.2 智慧养老数据预处理

In [15]:
# 加载中文停用词词典
with open('stopwords.txt', encoding='utf-8') as f:  
    stopwords = set(line.strip() for line in f)  # 创建一个停用词集合  
  
def clean_text(text):  
    # 去除非中文字符和特定词汇  
    text = remove_nums_and_words(text)  
      
    # 使用jieba分词并过滤停用词  
    words = jieba.lcut(text)  
    filtered_words = [w for w in words if w not in stopwords and w not in ['[', ']']]  
    return ' '.join(filtered_words)  
  
def remove_nums_and_words(text):  
    # 去除非中文字符  
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)  
    # 去除特定词汇  
    words_to_remove = ['关于', '人民政府', '印发']  
    for word in words_to_remove:  
        nonums = nonums.replace(word, '')  
    return nonums  
  
# 假设 df 是一个 pandas DataFrame，且 '标题' 列包含标题文本  
test = df['标题'][0]  
cleaned_text = clean_text(test)[:100]  # 截取前100个字符  
print(cleaned_text)

黑龙江省 办公厅 推进 养老 服务 发展 实施 意见


In [16]:
# 数据类型转换和处理
df['标题'] = df['标题'].astype(str)
df['标题分词'] = df['标题'].apply(remove_nums_and_words)
df['标题分词'] = df['标题分词'].apply(clean_text)
df['标题分词'] = df['标题分词'].apply(lambda x: x.split())
df

,序号,标题,时效性,效力位阶,制定机关,发文字号,公布日期,施行日期,批准部门,批准日期,原文链接,标题分词
0,地方规范性文件_672,黑龙江省人民政府办公厅关于推进养老服务发展的实施意见,现行有效,地方规范性文件,黑龙江省民政厅,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/b18e6be8f51692a1cbd...,"[黑龙江省, 办公厅, 推进, 养老, 服务, 发展, 实施, 意见]"
1,地方规范性文件_673,广西壮族自治区大数据发展局关于全面推进数字化转型发展的意见,现行有效,地方规范性文件,广西壮族自治区大数据发展局,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/22f0c8beef6908e6866...,"[广西壮族自治区, 数据, 发展局, 推进, 数字化, 转型, 发展, 意见]"
2,地方规范性文件_674,中共无锡市委、无锡市人民政府印发《＜关于加快推进数字经济高质量发展的实施意见＞＜关于加快推进...,现行有效,地方规范性文件,无锡市人民政府,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/42b6a5f466eaa99be8e...,"[中共, 无锡市委, 无锡市, 加快, 推进, 数字, 经济, 高质量, 发展, 实施, 意..."
3,地方规范性文件_675,贵阳市民政局关于推进贵阳贵安养老服务高质量发展的实施意见,现行有效,地方规范性文件,贵阳市民政局,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/e67d28d08fea8084a66...,"[贵阳市, 民政局, 推进, 贵阳, 贵安, 养老, 服务, 高质量, 发展, 实施, 意见]"
4,地方规范性文件_676,廊坊市人民政府办公室关于加快推进养老服务体系建设的实施意见,现行有效,地方规范性文件,廊坊市人民政府,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/909deb5c2273a94ef8b...,"[廊坊市, 办公室, 加快, 推进, 养老, 服务体系, 建设, 实施, 意见]"
...,...,...,...,...,...,...,...,...,...,...,...,...
2777,地方工作文件_15,郑州市人民政府关于印发郑州市数字政府建设行动方案的通知,现行有效,地方工作文件,郑州市人民政府,郑政〔2023〕25号,2023.12.25,2023.12.25,NaN,NaN,https://www.pkulaw.com/lar/b7908e42b1682d8bafd...,"[郑州市, 郑州市, 数字, 政府, 建设, 方案, 通知]"
2778,部门规章_1,产业结构调整指导目录(2024年本),尚未施行,部门规章,国家发展和改革委员会(含原国家发展计划委员会、原国家计划委员会),中华人民共和国国家发展和改革委员会令第7号,2023.12.27,2024.02.01,NaN,NaN,https://www.pkulaw.com/chl/d8513a2dd8e4e297bdf...,"[产业, 结构调整, 指导, 目录, 年本]"
2779,地方工作文件_14,山西省人民政府办公厅关于印发山西省数字政府建设规划(2023―2025年)的通知,现行有效,地方工作文件,山西省人民政府,晋政办发〔2023〕82号,2023.12.28,2023.12.28,NaN,NaN,https://www.pkulaw.com/lar/cf041968e90d5c7d666...,"[山西省, 办公厅, 山西省, 数字, 政府, 建设, 规划, 通知]"
2780,地方工作文件_13,山东省人民政府办公厅关于印发《山东省养老服务高质量发展三年行动计划(2024—2026年)》的通知,现行有效,地方工作文件,山东省人民政府,鲁政办字〔2023〕207号,2023.12.30,2023.12.30,NaN,NaN,https://www.pkulaw.com/lar/d0f5a014d1a25e2fc83...,"[山东省, 办公厅, 山东省, 养老, 服务, 高质量, 发展, 三年, 行动计划, 通知]"


## 2.3 LDA模型训练

In [17]:
# 创建词典和语料库
dictionary = corpora.Dictionary(df['标题分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['标题分词']]     # 根据分词结果创建语料库

In [18]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=6, id2word=dictionary, passes=15)

In [19]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.083*"养老" + 0.066*"服务" + 0.047*"通知" + 0.038*"发展" + 0.029*"居家"')
(1, '0.076*"养老" + 0.069*"意见" + 0.063*"实施" + 0.045*"服务" + 0.044*"办公室"')
(2, '0.053*"通知" + 0.038*"委员会" + 0.024*"健康" + 0.022*"工作" + 0.020*"信息化"')
(3, '0.099*"通知" + 0.062*"发展" + 0.051*"规划" + 0.041*"办公室" + 0.027*"十四五"')
(4, '0.060*"通知" + 0.022*"工程" + 0.022*"民生" + 0.021*"发展" + 0.020*"天津市"')
(5, '0.095*"通知" + 0.053*"发展" + 0.053*"建设" + 0.051*"规划" + 0.039*"办公室"')


In [20]:
# 查看分词结果
df['标题分词'].iloc[0]

['黑龙江省', '办公厅', '推进', '养老', '服务', '发展', '实施', '意见']

In [21]:
# 打印第一条主题和得分
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9069306254386902	 
Topic: 0.076*"养老" + 0.069*"意见" + 0.063*"实施" + 0.045*"服务" + 0.044*"办公室" + 0.033*"办公厅" + 0.030*"加快" + 0.029*"提升" + 0.027*"通知" + 0.027*"实施方案"

Score: 0.01874692365527153	 
Topic: 0.083*"养老" + 0.066*"服务" + 0.047*"通知" + 0.038*"发展" + 0.029*"居家" + 0.024*"实施方案" + 0.024*"办公室" + 0.023*"高质量" + 0.023*"推进" + 0.021*"社区"

Score: 0.018619593232870102	 
Topic: 0.095*"通知" + 0.053*"发展" + 0.053*"建设" + 0.051*"规划" + 0.039*"办公室" + 0.032*"十四五" + 0.021*"服务体系" + 0.019*"养老" + 0.016*"体系" + 0.015*"实施方案"

Score: 0.018597368150949478	 
Topic: 0.099*"通知" + 0.062*"发展" + 0.051*"规划" + 0.041*"办公室" + 0.027*"十四五" + 0.020*"一小" + 0.020*"一老" + 0.019*"解决方案" + 0.019*"整体" + 0.015*"重庆市"

Score: 0.018565015867352486	 
Topic: 0.060*"通知" + 0.022*"工程" + 0.022*"民生" + 0.021*"发展" + 0.020*"天津市" + 0.019*"贵州省" + 0.018*"办公室" + 0.017*"生活" + 0.017*"服务业" + 0.016*"委"

Score: 0.018540440127253532	 
Topic: 0.053*"通知" + 0.038*"委员会" + 0.024*"健康" + 0.022*"工作" + 0.020*"信息化" + 0.017*"卫生" + 0.016*"厅" + 0.013*"民政厅" + 0.013*"

In [10]:
# 从DataFrame中提取“标题分词”列的值
documents = df['标题分词'].values

In [11]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(0, 0.018574096), (1, 0.9070734), (2, 0.01856327), (3, 0.018543059), (4, 0.018553153), (5, 0.018693024)]

Document 2:
[(0, 0.28476107), (1, 0.64101654), (2, 0.018524297), (3, 0.018577231), (4, 0.018531159), (5, 0.018589742)]

Document 3:
[(1, 0.8682899), (5, 0.108648114)]

Document 4:
[(0, 0.013915899), (1, 0.78604615), (2, 0.013931183), (3, 0.013893755), (4, 0.013910609), (5, 0.1583024)]

Document 5:
[(0, 0.016690172), (1, 0.71143496), (2, 0.016725183), (3, 0.016732417), (4, 0.016674684), (5, 0.22174262)]

Document 6:
[(0, 0.012853223), (1, 0.01300363), (2, 0.4849544), (3, 0.013036171), (4, 0.46303633), (5, 0.0131162675)]

Document 7:
[(0, 0.018620057), (1, 0.76901954), (2, 0.018570578), (3, 0.018570628), (4, 0.15655199), (5, 0.018667225)]

Document 8:
[(0, 0.015315805), (1, 0.36271793), (2, 0.29186594), (3, 0.015340035), (4, 0.29948032), (5, 0.0152800055)]

Document 9:
[(0, 0.021341795), (1, 0.89508843), (2, 0.020856606), (3, 0.020854617), (4, 0.020873582), (5, 0.0209849

## 2.4 可视化

In [12]:
# 可视化及界面显示
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) # 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
pyLDAvis.display(lda_vis)